In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt

import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/iyeonsu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/iyeonsu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/iyeonsu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
df_train = pd.read_csv("train (1).csv") # 학습용 데이터
df_test = pd.read_csv("submission (1).csv") # 테스트 데이터(제출파일의 데이터)

In [4]:
df_train.head() # 학습용 데이터 살펴보기

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.0,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.0,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.0,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.0,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.0,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,True


In [5]:
# 중복 데이터 제거 
df_train = df_train.drop_duplicates()
df_train.shape

(55780, 29)

In [6]:
def make_country_list(df1, df2):
    df_all = pd.concat([df1, df2.drop('id', axis = 1)])

    # null값은 모두 'unknown'으로 전처리
    # 추후에 다시 0으로 처리
    df_all['customer_country'] = df_all['customer_country'].fillna('unknown')
    df_all['customer_country'] = df_all['customer_country'].str.lower()
    # '/' 기준 분리 후 맨 뒤를 국적으로 별도 변수 생성
    def country_split(text):
        text = text.split('/')
        return text[-1]
    country_list = df_all['customer_country'].apply(country_split)
    # 공백 제거
    country_list = country_list.str.strip()

    # 20개 이상 국가만 사용: 550 -> 83
    country_list = list(country_list.value_counts()[:83].index)
    country_list.remove('')
    # indianapolis 는 US 도시
    country_list.remove('india')
    country_list.append('india')
    return country_list

In [7]:
country_list = make_country_list(df_train, df_test)

In [8]:
def make_weight_dic(df):
    df['expected_timeline'] = df['expected_timeline'].fillna('unknown')
    
    delete_data = ['less than 3 months','3 months ~ 6 months','more than a year','9 months ~ 1 year',
               '6 months ~ 9 months','less than 6 months']

    # 명사와 동사만 남기는 전처리
    def text_clean(text):
        tokens = word_tokenize(text)
        # 불용어 제거
        stop_word = set(stopwords.words('english'))
        tokens = [t for t in tokens if t not in stop_word]
        pos_tags = pos_tag(tokens)

        nouns = [word for word, pos in pos_tags if pos.startswith('N')]
        verbs = [word for word, pos in pos_tags if pos.startswith('V')]

        all = nouns + verbs

        return all


    # 실패/성공에 대한 결과가 있어야 하므로 train만 적용
    # 영업전환 실패한 고객들의 timeline
    timeline_f = []
    for i in df[df['is_converted'] == False]['expected_timeline']:
        if i not in delete_data:
            text = text_clean(i)
            timeline_f.extend(text)

    # 영업전환 성공한 고객들의 timeline
    timeline_t = []
    for i in df[df['is_converted']]['expected_timeline']:
        if i not in delete_data:
            text = text_clean(i)
            timeline_t.extend(text)

    count_f = Counter(timeline_f).most_common()
    count_f = pd.DataFrame(count_f, columns = ['word', 'count'])

    count_t = Counter(timeline_t).most_common()
    count_t = pd.DataFrame(count_t, columns = ['word', 'count'])
    
    # 가장 빈도수 높은 단어 10개(실패)
    dic_ex_f = count_f.head(10).set_index('word').to_dict()['count']

    # 가장 빈도수 높은 단어 10개(성공)
    # unknown 제외
    dic_ex = count_t.head(10).set_index('word').to_dict()['count']

    # top10 단어들로 가중치 주기
    dic_ex_f['client'] = -10
    dic_ex_f['details'] = -9
    dic_ex_f['etc'] = -8
    dic_ex_f['followed'] = -7
    dic_ex_f['requirement'] = -6
    dic_ex_f['shared'] = -5
    dic_ex_f['system'] = -4
    dic_ex_f['customer'] = -3
    dic_ex_f['hence'] = -2
    dic_ex_f['call'] = -1

    dic_ex['demo'] = 10
    dic_ex['client'] = 9
    dic_ex['customer'] = 8
    dic_ex['shared'] = 7
    dic_ex['details'] = 6
    dic_ex['call'] = 5
    dic_ex['send'] = 4
    dic_ex['discussed'] = 3
    dic_ex['quote'] = 2
    dic_ex['followed'] = 1

    dic_all = {}

    for key, value in dic_ex_f.items():
        dic_all[key] = value

    for key, value in dic_ex.items():
        dic_all[key] = dic_all.get(key, 0) + value

    return dic_all

In [9]:
dic_all = make_weight_dic(df_train)

In [10]:
def preprocessing(df):
    # product_category
    product_dict = {}
    product_dict['HVAC/ESS'] = ['control', 'ventilation', 'vrf', 'multi-split', 'single-split', 'chiller', 'heating']
    product_dict['commercial display'] = ['oled signage', 'led signage', 'video wall signage', 'interactive signage',
                                        'high brightness signage', 'special signage', 'standard signage', 'hotel tv', 'hospital tv', 'software solution',
                                        'signage care solution', 'webos', 'procentric', 'one quick', 'interactive digital board']
    product_dict['it products'] = ['monitor', 'laptop', 'projector', 'cloud device', 'medical display']

    def original_category(text):
        for key in product_dict.keys():
            if text in product_dict[key]:
                return key

    # site 속 카테고리 기준 original 카테고리 변수 생성
    df['category'] = df['product_category'].apply(original_category)

    # ',' 포함한 데이터는 integrated solution으로 solution 카테고리로 처리
    df.loc[df['product_category'].str.contains(',', na = False), 'category'] = 'solution'

    # 임시로 nan으로 처리
    df['category'] = df['category'].fillna('nan')

    # product_dict로 처리되지 않은 카테고리 확인
    df[df['category'] == 'nan']['product_category'].value_counts()

    # 카테고리 입력 함수
    def category_input(text, input):
        df.loc[(df['product_category'].str.contains(text, na = False))&(df['category'] == 'nan'), 'category'] = input

    # 언어 번역
    def category_lang(text, input):
        df.loc[df['product_category'].str.contains(text, na = False), 'product_category'] = input

    # teto ou cassete inverter 번역 => 'ceiling or inverter cassette' => hvac/ess
    category_lang('teto ou cassete inverter', 'ceiling or inverter cassette')

    # rac => residential air conditioner => hvac/ess
    category_lang('rac', 'residential air conditioner')

    # ar condicionado residencial => residential air conditioner => hvac/ess
    category_lang('ar condicionado residencial', 'residential air conditioning')

    # outros => others
    category_lang('outros', 'others')

    # technical support, lg customer care program => support
    category_input('technical support', 'support')
    category_input('lg customer care program', 'support')

    # scroll compressor => compressor/motor
    category_input('scroll compressor', 'compressor/motor')

    # robots => robot
    category_input('robots', 'robot')


    # ogrzewanie (pompy ciepła) => heating(heat pumps)
    category_lang('ogrzewanie', 'heating(heat pumps)')

    # aire acondicionado residencial => residential air conditioning
    category_lang('aire acondicionado residencial', 'residential air conditioning')
    # led 顯示屏 => led display
    category_lang('led 顯示屏', 'led display')
    # isıtma => heating
    category_lang('isıtma', 'heating')
    # lainnya => other
    category_lang('lainnya', 'other')
    # calefacción => heating
    category_lang('calefacción', 'heating')
    # 互動式顯示屏 => interactive display
    category_lang('互動式顯示屏', 'interactive display')
    # 標準顯示屏 => standard display
    category_lang('標準顯示屏', 'standard display')
    # điều hòa trung tâm vrf => vrf central air conditioner
    category_lang('điều hòa trung tâm vrf', 'vrf central air conditioner')
    # soğutucu => cooler
    category_lang('soğutucu', 'cooler')
    # تكييف وتبريد => conditioning and cooling
    category_lang('تكييف وتبريد', 'conditioning and cooling')
    # 特別顯示屏 => special display
    category_lang('特別顯示屏', 'special display')
    # מזגנים למקום מגורים => residential air conditioner
    category_lang('מזגנים למקום מגורים', 'residential air conditioner')
    # เครื่องปรับอากาศเผื่อที่อยู่อาศัย => residential air conditioner
    category_lang('เครื่องปรับอากาศเผื่อที่อยู่อาศัย', 'residential air conditioner')
    # חימום => heating
    category_lang('חימום', 'heating')
    # تكييفات => air conditioner
    category_lang('تكييفات', 'air conditioner')
    # điều hòa cục bộ => local air conditioning
    category_lang('điều hòa cục bộ', 'local air conditioning')
    # 醫院電視 => hospital tv
    category_lang('醫院電視', 'hospital tv')
    # 高亮度顯示屏 => high brightness display
    category_lang('高亮度顯示屏', 'high brightness display')
    # 軟體 => software
    category_lang('軟體', 'software')
    # פיצול מרובה => multi split
    category_lang('פיצול מרובה', 'multi split')
    # 酒店電視 => hotel tv
    category_lang('酒店電視', 'hotel tv')
    # حلول التدفئة => heating solution
    category_lang('حلول التدفئة', 'heating solution')
    # אחר => other
    category_lang('אחר', 'other')
    # مبرد (تشيلر) => chiller
    category_lang('مبرد', 'chiller')
    # ฯลฯ => etc.
    category_lang('ฯลฯ', 'etc.')
    # điều hòa gia dụng => household air conditioner
    category_lang('điều hòa gia dụng', 'household air conditioner')
    # khác => other
    category_lang('khác', 'other')
    # otros => others
    category_lang('otros', 'others')
    # pendingin => cooler
    category_lang('pendingin', 'cooler')
    # ac rumah => home air conditioning
    category_lang('ac rumah', 'home air conditioning')
    # climatiseur résidentiel => residential air conditioner
    category_lang('climatiseur résidentiel', 'residential air conditioner')

    # it ptoducts
    it_products = ['pc', 'medical display', '28mq780', 'medical', 'monitor',
                'radiology displays', 'bu50nst', 'notebook']
    it_text = '|'.join(it_products)
    category_input(it_text, 'it products')

    # hvac/ess
    hvac_products = ['all lg vrf systems', 'multi', 'a thermodynamic water heater',
                    'residential', 'heating', 'chiller', 'condition', 'vrf',
                    'cooler', 'split','energy storage system', 'cac', 'single cac',
                    'system ac', 'ceiling or inverter cassette', 'residential air conditioner',
                    'multi inverter', 'residential air conditioning', 'ess', 'drv']
    hvac_text = '|'.join(hvac_products)
    category_input(hvac_text, 'hvac/ess')

    # commercial display
    display_products = ['ur640', 'signage', 'virtual production', 'commercial tv',
                        'videowall','43us660h0sd.awz','ledallinone','onequick',
                        'led display','education createboard', '.awz','allinone',
                        'leadallin','tv','fhd series', 'bwz', 'interactive display',
                        'one quick', 'series', 'aio', 'led','lsca039','43us660h',
                        '55vm5e', 'pro centric', 'gscd100','standard', 'lg magnit',
                        '86uh5f', '49vl5f','98uh5e', '55vm5j-h', '55tc3d', '49vl5g-m', '55svh7f-a', 'hospitality', 'laec15',
                        'retaildigital','gscd046', 'gsca046', 'collaboration displays', 'tr3', 'taa lcd lfd displays',
                        'window facing display', 'special display', 'hoteleria_us670h', 'software',
                        'laec015', 'high brightness display','videwall', 'idb', 'one:quick',
                        'high brightness', 'video wall', 'pro:centric', 'commercial display',
                        'lg paradise air solution'

                        ]
    display_text = '|'.join(display_products)
    category_input(display_text, 'commercial display')

    # product_dict로 처리되지 않은 카테고리 확인
    df[df['category'] == 'nan']['product_category'].value_counts()

    # 카테고리 입력 함수
    # subcategory 기준
    def sub_input(text, input):
        df.loc[(df['product_subcategory'].str.contains(text, na = False))&(df['category'] == 'nan'), 'category'] = input

    # product_subcategory, modelname만 적혀있는것도 큰 카테고리로 분류
    df['product_subcategory'] = df['product_subcategory'].str.lower()

    # จอภาพสำหรับการตรวจสอบทางคลินิก => monitor for clinical monitoring -> it products
    # จอภาพเพื่อการวินิจฉัย => diagnostic monitor
    # 其他 => other
    df.loc[df['product_subcategory'].str.contains('其他', na = False), 'category'] = 'other'
    # monitor => it products
    it_sub = ['monitor', 'medical', 'จอภาพสำหรับการตรวจสอบทางคลินิก', 'จอภาพเพื่อการวินิจฉัย',
            'cloud device', 'digital x-ray detectors', 'thin clients',
            'all projectors', 'laptops', 'probeam', 'zero clients']
    it_text = '|'.join(it_sub)
    sub_input(it_text, 'it products')

    # hvac/ess
    hvac_sub = ['all lg vrf systems', 'multi', 'a thermodynamic water heater',
                    'residential', 'heating', 'chiller', 'condition', 'vrf',
                    'cooler', 'split','energy storage system', 'cac', 'single cac',
                    'system ac', 'ess', '3.0 tr -1 nos. cassette']
    hvac_text = '|'.join(hvac_sub)
    sub_input(hvac_text, 'hvac/ess')

    # commercial display
    display_sub = ['pro:centric', 'signage', 'one:quick' ,'one-quick', 'webos box',
                'interactive digital board', 'tr3dj series', 'tr3bg series',
                'lg ops player', '65tr3bf', 'idb', 'lg smart cam pro','65tr3dj', 'supersign cms']
    display_text = '|'.join(display_sub)
    sub_input(display_text, 'commercial display')

    # 처리되지 않은 subcategory 확인
    df[df['category'] == 'nan']['product_subcategory'].value_counts()

    # 카테고리 입력 함수
    # modelname 기준
    def model_input(text, input):
        df.loc[(df['product_modelname'].str.contains(text, na = False))&(df['category'] == 'nan'), 'category'] = input

    # modelname 카테고리 분류

    it_model = ['UltraFine', '28MQ780', 'Ergo Dual', '21HQ513D', 'UltraWide', '32UN880',
                '31HN713D', '14HQ701G-BP', '38CL950P', 'Radiology']
    it_text = '|'.join(it_model)
    model_input(it_text, 'it products')

    # hvac/ess
    hvac_model = ['all lg vrf systems', 'multi', 'a thermodynamic water heater',
                    'residential', 'heating', 'chiller', 'condition', 'vrf',
                    'cooler', 'split','energy storage system', 'cac', 'single cac',
                    'system ac']
    hvac_text = '|'.join(hvac_model)
    model_input(hvac_text, 'hvac/ess')

    # commercial display
    display_model = ['43HT3WJ', '55CT5WJ', 'SC-00DA', 'LG SuperSign CMS', '65EP5G OLED Pro',
                    '34WN780', 'IDB', 'LSVP']
    display_text = '|'.join(display_model)
    model_input(display_text, 'commercial display')

    # 처리되지 않은 subcategory 확인
    df[df['category'] == 'nan']['product_modelname'].value_counts()

    # 처리되지 않은 것 모두 other로 통일
    df[df['category'] == 'nan']['product_category'].value_counts()

    df.loc[df['category'] == 'nan', 'category'] = 'other'

    # product 변수 얼마나 작성하였는지
    df['product_category'] = df['product_category'].fillna('unknown')
    df['product_subcategory'] = df['product_subcategory'].fillna('unknown')
    df['product_modelname'] = df['product_modelname'].fillna('unknown')

    df['product_count'] = 0
    for i, row in df.iterrows():
        count = 0
        if row['product_category'] != 'unknown':
            count += 1
        if row['product_subcategory'] != 'unknown':
            count += 1
        if row['product_modelname'] != 'unknown':
            count += 1
        df.loc[i, 'product_count'] = count

    # '_' -> 공백으로 처리
    df['expected_timeline'] = df['expected_timeline'].str.replace('_', ' ')

    # null값 unknown
    df['expected_timeline'] = df['expected_timeline'].fillna('unknown')

    # 단어가 들어있는 비중에 따라 가중치
    df['timeline_count'] = ''
    for i, row in df.iterrows():
        score = 0
        if row['expected_timeline'] == 'unknown':
            pass
        else:
            for key in dic_all.keys():
                if key in row['expected_timeline']:
                    score += dic_all[key]

        df.loc[i, 'timeline_count'] = score

    delete_data = ['less than 3 months','3 months ~ 6 months','more than a year','9 months ~ 1 year',
                   '6 months ~ 9 months','less than 6 months']

    df.loc[~df['expected_timeline'].isin(delete_data), 'expected_timeline'] = 0

    df['historical_existing_cnt'] = df['historical_existing_cnt'].fillna(0)

    # other 통일
    df.loc[df['customer_job'].str.contains('other', na = False), 'customer_job'] = 'others'

    # '_' 공백으로 변경
    df['customer_job'] = df['customer_job'].str.replace('_', ' ')

    # null값 0으로 처리
    df['customer_job'] = df['customer_job'].fillna('others')


    def job_categorize(text):
        if 'accounting' in text:
            return 'accounting'
        elif 'administrative' in text:
            return 'administrative'
        elif 'arts and design' in text:
            return 'arts and design'
        elif 'business development' in text:
            return 'business development'
        elif 'community and social services' in text:
            return 'community and social services'
        elif 'consulting' in text:
            return 'consulting'
        elif 'curation' in text:
            return 'curation'
        elif 'education' in text:
            return 'education'
        elif 'engineering' in text:
            return 'engineering'
        elif 'entrepreneurship' in text:
            return 'entrepreneurship'
        elif 'finance' in text:
            return 'finance'
        elif 'healthcare services' in text:
            return 'healthcare services'
        elif 'human resources' in text:
            return 'human resources'
        elif 'information technology' in text:
            return 'information technology'
        elif 'legal' in text:
            return 'legal'
        elif 'marketing' in text:
            return 'marketing'
        elif 'media and communication' in text:
            return 'media and communication'
        elif 'military and protective services' in text:
            return 'military and protective services'
        elif 'operations' in text:
            return 'operations'
        elif 'product management' in text:
            return 'product management'
        elif 'program and project management' in text:
            return 'program and project management'
        elif 'purchasing' in text:
            return 'purchasing'
        elif 'quality assurance' in text:
            return 'quality assurance'
        elif 'real estate' in text:
            return 'real estate'
        elif 'research' in text:
            return 'research'
        elif 'sales' in text:
            return 'sales'
        elif 'support' in text:
            return 'support'
        else:
            return 'other'

    df['customer_job'] = df['customer_job'].apply(job_categorize)

    # '-' 제거 및 '/' -> ','으로 처리
    df['customer_type'] = df['customer_type'].str.replace('-', ' ')
    df['customer_type'] = df['customer_type'].str.replace('/', ',')

    # ',' 뒤에만 공백이 남도록 전처리
    df['customer_type'] = df['customer_type'].apply(lambda x: re.sub(r'\s*,\s*', ', ', x) if isinstance(x, str) else x)

    # other 통일
    df.loc[df['customer_type'].str.contains('Other', na = False), 'customer_type'] = 'other'

    # etc도 other로 통일
    df['customer_type'] = df['customer_type'].str.replace('Etc.', 'other')

    # homeowner 통일
    df.loc[df['customer_type'].str.contains('Home', na = False), 'customer_type'] = 'Homeowner'

    # 1. null값 모두 other로 변경해서 처리
    df['customer_type'] = df['customer_type'].fillna('other')

    # other를 0으로 전처리
    df['customer_type'] = df['customer_type'].replace('other', 0)

    # installer -> installer, contractor
    df.loc[df['customer_type'].str.contains('Installer', na = False), 'customer_type'] = 'Installer, Contractor'

    # distributor -> dealer, distributor
    df.loc[df['customer_type'].str.contains('Distributor', na = False), 'customer_type'] = 'Dealer, Distributor'

    # consultant -> architect, consultant
    df.loc[df['customer_type'].str.contains('Consultant', na = False), 'customer_type'] = 'Architect, Consultant'

    # 위에 false만 있는 값을 모두 모아 1로 묶음
    df['customer_type'] = df['customer_type'].replace(['Corporate', 'Dealer, Distributor', 'System Integrator', 'Technician', 'Engineer', 'Manager, Director', 'Developer', 'End user', 'HVAC Engineer', 'Reseller', 'Software, Solution Provider', 'Technical Assistant', 'Commercial end user', 'Interior Designer', 'Administrator'], 1)

    # 이건 모두 null값 0으로 처리
    columns = ['id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver']

    for col in columns:
        df[col] = df[col].fillna(0)

    # idit_all : id = 1, it =2, 결측치 = 0

    # 'idit_all' 열 초기화
    df['idit_all'] = 0

    # 'id_strategic_ver'에서 1 -> 1
    df.loc[df['id_strategic_ver'] == 1, 'idit_all'] = 1

    # 'it_strategic_ver'에서 1 -> 2
    df.loc[df['it_strategic_ver'] == 1, 'idit_all'] = 2

    # 나머지는 0으로 저장
    df['idit_all'].fillna(0)

    # id_strategic_ver, it_strategic_ver, idit_strategic_ver 드롭
    df.drop(columns=['id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver'], inplace=True)

    def preprocess_country(text):
        # 나라 이름으로 전처리
        for country in list(country_list):
            if country in text:
                # print(country, text)
                return country
        return 'other'

    # 'customer_country' 열에 대해 전처리 적용
    df['customer_country'] = df['customer_country'].fillna('unknown')
    df['customer_country'] = df['customer_country'].str.lower()
    df['customer_country'] = df['customer_country'].apply(preprocess_country)

    # customer_country.1 드롭
    df.drop(columns=['customer_country.1'], inplace=True)

    # 데이터를 문자열로 변환하는 함수
    def convert_to_string(value):
        return str(value) if value is not None else ''

    def preprocess_inquiry(text):
        # 문자열로 변환
        text = convert_to_string(text)

        # 소문자로 변환
        text = text.lower()

        # 특수 문자 제거
        text = re.sub(r'[^a-zA-Z\s]', '', text)

        # 중복된 값 처리
        if 'technical' in text and text != 'technical support':
            return 'technical'
        if 'quotation' in text:
            return 'quotation'
        if 'sales' in text:
            return 'sales'
        if 'other' in text or 'etc' in text:
            return 'other'
        for inquiry in ['customer suggestions', 'nan', 'technical support', 'partnership', 'distributorship', 'demo', 'services', 'product information', 'trainings']:
            if inquiry in text:
                return text
        return 'others'

    # 'inquiry_type' 열에 대해 전처리 적용
    df['inquiry_type'] = df['inquiry_type'].apply(preprocess_inquiry)

In [11]:
preprocessing(df_train)
preprocessing(df_test)

In [12]:
# 파생변수 생성 함수 
def generate_feature(df):
    # bant_submit
    df['bant_submit_count'] = df['bant_submit'].apply(lambda x: 1 if x == 0 else 0)

    # com_reg_ver_win_rate
    df['com_reg_count'] = df['com_reg_ver_win_rate'].apply(lambda x: 1 if x > 0.04 else 0)

    # customer_idx
    idx_count = df['customer_idx'].value_counts()
    df['idx_count'] = df['customer_idx'].apply(lambda x: 1 if x in idx_count[idx_count>1].index else 0)

    # lead_desc_length
    df['lead_log'] = df['lead_desc_length'].apply(lambda x: np.log(x))
    df['lead_count'] = df['lead_log'].apply(lambda x: 1 if x > 3.367296 else 0)
    # 전처리 과정에서 일단 lead_desc_length를 제거하진 않겠습니다

    # historical_existing_cnt
    df['enterprise_count'] = 0
    df.loc[(df['enterprise'] == 'Enterprise')&(df['historical_existing_cnt']!=0), 'enterprise_count'] = 1

    # enterprise, SMB 둘 다 있는 회사명에 가중치 
    enterprise_2 = df.groupby('customer_idx')['enterprise'].nunique()
    idx = enterprise_2[enterprise_2==2].index
    df.loc[df['customer_idx'].isin(idx), 'enterprise_weight'] = 1
    df.loc[~df['customer_idx'].isin(idx), 'enterprise_weight'] = 0


In [17]:
generate_feature(df_train)
generate_feature(df_test)

In [20]:
df_train['enterprise_weight'].value_counts()

enterprise_weight
0.0    43061
1.0    12719
Name: count, dtype: int64

In [21]:
df_test['enterprise_weight'].value_counts()

enterprise_weight
0.0    4912
1.0     359
Name: count, dtype: int64

In [22]:
df_train.head(5)

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,customer_job,lead_desc_length,...,product_count,timeline_count,idit_all,bant_submit_count,com_reg_count,idx_count,lead_log,lead_count,enterprise_count,enterprise_weight
0,1.0,philippines,AS,0.066667,32160,End Customer,Enterprise,0.0,purchasing,62,...,1,0,0,0,1,1,4.127134,1,0,0.0
1,1.0,philippines,AS,0.066667,23122,End Customer,Enterprise,12.0,media and communication,96,...,1,0,0,0,1,1,4.564348,1,1,0.0
2,1.0,india,AS,0.088889,1755,End Customer,Enterprise,144.0,engineering,56,...,1,0,0,0,1,1,4.025352,1,1,0.0
3,1.0,india,AS,0.088889,4919,End Customer,Enterprise,0.0,entrepreneurship,44,...,1,0,0,0,1,0,3.784190,1,0,0.0
4,1.0,india,AS,0.088889,17126,"Specifier, Influencer",Enterprise,0.0,consulting,97,...,1,0,0,0,1,0,4.574711,1,0,0.0


In [12]:
df_test.head(5)

,id,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,customer_job,...,category,product_count,timeline_count,idit_all,bant_submit_count,com_reg_count,idx_count,lead_log,lead_count,enterprise_count
0,19844,0.00,brazil,ID,0.073248,47466,End Customer,Enterprise,53.0,consulting,...,other,0,0,0,1,1,1,1.098612,0,1
1,9738,0.25,united states,IT,NaN,5405,End Customer,SMB,0.0,other,...,other,0,0,0,1,0,1,3.135494,0,0
2,8491,1.00,u.a.e,ID,NaN,13597,"Specifier, Influencer",SMB,0.0,information technology,...,commercial display,3,0,0,1,0,1,4.927254,1,0
3,19895,0.50,united states,ID,0.118644,17204,0,Enterprise,0.0,sales,...,commercial display,1,0,0,1,1,0,1.098612,0,0
4,10465,1.00,brazil,ID,0.074949,2329,End Customer,Enterprise,2.0,engineering,...,commercial display,3,0,1,1,1,1,4.672829,1,1


In [23]:
# 전처리 파일 저장
df_train.to_csv("train_final.csv", index=False)
df_test.to_csv("submission_final.csv", index=False)